In this notebook, we will calculate features using Trotterization with IBM's QPU.

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
# This is necessary only when running the notebook in the docker container
%cd fourier_learning_ibm/

/root/workdir/fourier_learning_ibm


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# GPU check
!nvidia-smi -L

GPU 0: NVIDIA A100-PCIE-40GB (UUID: GPU-3fac7503-bb55-35cd-f566-c919eaa55798)
GPU 1: NVIDIA A100-PCIE-40GB (UUID: GPU-bf03abcd-ef36-6733-da57-460c8eb9cab4)
GPU 2: NVIDIA A100-PCIE-40GB (UUID: GPU-bc01f8f3-0939-f2c0-6998-dd49d0360463)
GPU 3: NVIDIA A100-PCIE-40GB (UUID: GPU-b7aeac00-b332-1045-eebc-bb5653ac8402)


In [4]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import pandas as pd
import os
from heisenberg import (
    HeisenbergModel,
    get_graph,
    get_positions,
    # get_initial_layout,
    # get_prob0,
)
from setup import setup_backend
from helper_features import run_job, get_features
from qiskit import transpile
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2 as Sampler, Batch
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
import pickle
import networkx as nx
from datetime import datetime, timezone
import json
import math

In [5]:
import warnings

warnings.filterwarnings("ignore")

### Config backend

In [6]:
# density matrix simulator
backend_qpu, backend_dm_noiseless, backend_dm_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="density_matrix"
)

print()

# matrix product state simulator
backend_qpu, backend_mps_noiseless, backend_mps_noisy = setup_backend(
    qpu_name="ibm_marrakesh", device="GPU", method="matrix_product_state"
)

Using backend QPU: <IBMBackend('ibm_marrakesh')>

NoiseModel:
  Basis gates: ['cz', 'delay', 'for_loop', 'id', 'if_else', 'measure', 'reset', 'rz', 'switch_case', 'sx', 'x']
  Instructions with noise: ['id', 'measure', 'x', 'sx', 'cz', 'reset']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155]
  Specific qubit errors: [('id', (0,)), ('id', (1,)), ('id', (2

### Prepare parameters and load dataset

In [7]:
n_qubits = 40

C = 3
n_samples = 55
n_shots = 1000

if n_qubits <= 20:  # n_qubits <= 20
    threshold = 0.6

    # Load dataset (orient="records")
    if n_qubits == 20:  # 20 量子ビットの場合は、12 量子ビットの結果を使う
        df_n_step = pd.read_json(
            f"results/get_nstep/12Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    else:
        df_n_step = pd.read_json(
            f"results/get_nstep/{n_qubits}Q/threshold{threshold:.2f}/optimal_n_step.json",
            orient="records",
        )
    display(df_n_step)

    # Note that K = n_features-1.
    # n_features counts only the number of real features, also the one of t=0, i.e., constant term.
    # The number of all features, including imaginary numbers, is 2K+1.
    n_features = len(df_n_step)

    times = df_n_step["t"].values
    n_step_array = df_n_step["n_step"].values

else:  # n_qubits > 20
    threshold = None
    n_features = 12
    # 間隔 np.pi/C で等間隔な点を n_features 個生成
    times = np.linspace(0, np.pi / C * (n_features - 1), n_features)

    # n_qubits > 20 の場合は、n_step は 1 から max_n_step の整数値を取る
    if n_qubits == 32:
        max_n_step = 4
    elif n_qubits == 40:
        max_n_step = 5
    # まず 1 から max_n_step までの n_features 個の等間隔の数値を生成
    n_step_array = np.linspace(1, max_n_step, n_features)
    # 各値を四捨五入して整数に変換
    n_step_array = [round(x) for x in n_step_array]

print(f"n_qubits: {n_qubits}")
print(f"C: {C}")
print(f"threshold: {threshold}")
print(f"n_samples: {n_samples}")
print(f"n_shots: {n_shots}")
print(f"n_features: {n_features}")
print(f"times: {times}")
print(f"n_step_array: {n_step_array}")

# Load dataset (orient="records")
df_dataset = pd.read_json(
    f"results/get_dataset/{n_qubits}Q/dataset.json",
    orient="records",
)
all_Js = df_dataset["Js"].values
all_expected_values = df_dataset["expected_value"].values
n_samples = len(all_Js)
display(df_dataset)

n_qubits: 40
C: 3
threshold: None
n_samples: 55
n_shots: 1000
n_features: 12
times: [ 0.          1.04719755  2.0943951   3.14159265  4.1887902   5.23598776
  6.28318531  7.33038286  8.37758041  9.42477796 10.47197551 11.51917306]
n_step_array: [1, 1, 2, 2, 2, 3, 3, 4, 4, 4, 5, 5]


,sample,Js,expected_value,expected_value_diag,diff
0,0,"[0.027569343700000002, -0.0061509184, 0.036087...",0.913388,NaN,NaN
1,1,"[0.0198111334, -0.0073320913, 0.0388114779, 0....",1.057062,NaN,NaN
2,2,"[0.0059292635, 0.0295150582, 0.017082614700000...",1.009956,NaN,NaN
3,3,"[0.0255905961, -0.0079183276, 0.0148449159, 0....",1.115899,NaN,NaN
4,4,"[-0.0199742169, 0.0145559143, 0.0207255021, 0....",0.906205,NaN,NaN
5,5,"[-0.0313871088, 0.009725694600000001, 0.036654...",0.956448,NaN,NaN
6,6,"[0.0397126215, 0.0279129831, -0.00299143800000...",1.192622,NaN,NaN
7,7,"[0.0097647419, 0.0193609483, -0.0113959584, -0...",0.961146,NaN,NaN
8,8,"[-0.0277423153, -0.013743889300000001, -0.0138...",0.915409,NaN,NaN
9,9,"[-0.035715704, 0.0070571812, 0.0078699068, -0....",0.926034,NaN,NaN


# Create Fourier feature

## Trotter simulation

In [8]:
config = {
    "n_qubits": n_qubits,
    "n_samples": n_samples,
    "n_shots": n_shots,
    "n_features": n_features,
    "times": times,
    "all_Js": all_Js,
    "n_step_array": n_step_array,
    "backend": backend_qpu,  # This is important!!
    "all_expected_values": all_expected_values,
}

In [9]:
# For QPU, we need batch instance and job_ids instead of job instances
batch, _, job_ids = run_job(config, backend_qpu, sim_type="qpu")

In [13]:
path = f"results/fourier_feature_qpu/{n_qubits}Q"

In [11]:
# Save job_ids
with open(f"{path}/job_ids.txt", "w") as f:
    for job_id in job_ids:
        f.write(f"{job_id}\n")

In [12]:
batch.status()

'In progress, not accepting new jobs'

In [14]:
# Load job_ids
with open(f"{path}/job_ids.txt", "r") as f:
    job_ids = [job_id.strip() for job_id in f.readlines()]

# fetch successful jobs
jobs = []
error_count = 0
for i in range(n_samples):
    print(f"Checking job {i}/{n_samples}")
    job_id = job_ids[i]
    job = backend_qpu.service.job(job_id)

    if job.status() == "DONE":
        jobs.append(job)
    else:
        error_count += 1
        print(f"Status: {job.status()}")

Checking job 0/55
Checking job 1/55
Checking job 2/55
Checking job 3/55
Checking job 4/55
Checking job 5/55
Checking job 6/55
Checking job 7/55
Checking job 8/55
Checking job 9/55
Checking job 10/55
Checking job 11/55
Checking job 12/55
Checking job 13/55
Checking job 14/55
Checking job 15/55
Checking job 16/55
Checking job 17/55
Checking job 18/55
Checking job 19/55
Checking job 20/55
Checking job 21/55
Checking job 22/55
Checking job 23/55
Checking job 24/55
Checking job 25/55
Checking job 26/55
Checking job 27/55
Checking job 28/55
Checking job 29/55
Checking job 30/55
Checking job 31/55
Checking job 32/55
Checking job 33/55
Checking job 34/55
Checking job 35/55
Checking job 36/55
Checking job 37/55
Checking job 38/55
Checking job 39/55
Checking job 40/55
Checking job 41/55
Checking job 42/55
Checking job 43/55
Checking job 44/55
Checking job 45/55
Checking job 46/55
Checking job 47/55
Checking job 48/55
Checking job 49/55
Checking job 50/55
Checking job 51/55
Checking job 52/55
Che

In [15]:
features_df = get_features(config, jobs)
display(features_df)

Post-processing sample 0/55
Post-processing sample 1/55
Post-processing sample 2/55
Post-processing sample 3/55
Post-processing sample 4/55
Post-processing sample 5/55
Post-processing sample 6/55
Post-processing sample 7/55
Post-processing sample 8/55
Post-processing sample 9/55
Post-processing sample 10/55
Post-processing sample 11/55
Post-processing sample 12/55
Post-processing sample 13/55
Post-processing sample 14/55
Post-processing sample 15/55
Post-processing sample 16/55
Post-processing sample 17/55
Post-processing sample 18/55
Post-processing sample 19/55
Post-processing sample 20/55
Post-processing sample 21/55
Post-processing sample 22/55
Post-processing sample 23/55
Post-processing sample 24/55
Post-processing sample 25/55
Post-processing sample 26/55
Post-processing sample 27/55
Post-processing sample 28/55
Post-processing sample 29/55
Post-processing sample 30/55
Post-processing sample 31/55
Post-processing sample 32/55
Post-processing sample 33/55
Post-processing sample 3

,sample_id,f_0 Re,f_1 Re,f_1 Im,f_2 Re,f_2 Im,f_3 Re,f_3 Im,f_4 Re,f_4 Im,...,f_7 Im,f_8 Re,f_8 Im,f_9 Re,f_9 Im,f_10 Re,f_10 Im,f_11 Re,f_11 Im,expected_value
0,0,0.551,0.030461,-0.119675,-0.001360,-0.058448,0.005464,-0.068660,0.010443,-0.052230,...,-0.007505,-0.016591,0.007054,-0.019712,-0.005236,-0.008287,0.005597,-0.006555,0.006785,0.913388
1,1,0.579,0.090748,0.024141,0.049545,0.024602,0.051107,0.034410,0.044573,0.050579,...,0.010586,0.002931,0.011332,0.004586,0.006631,0.007608,0.000344,-0.001471,0.002416,1.057062
2,2,0.579,0.125985,0.004457,0.047058,-0.004747,0.045491,0.002138,0.042955,0.009739,...,-0.000838,0.024162,-0.005405,0.005903,0.012576,0.001453,0.012645,0.009955,0.000944,1.009956
3,3,0.662,-0.000749,-0.109236,0.005293,-0.057324,0.012184,-0.086813,0.016147,-0.068617,...,-0.010753,-0.004051,-0.014986,0.000691,-0.019223,0.000254,-0.008540,-0.001694,-0.008314,1.115899
4,4,0.625,0.045894,-0.121782,-0.009468,-0.088059,-0.036764,-0.082537,-0.006860,-0.070837,...,-0.005206,-0.017433,-0.004011,-0.021145,-0.007867,-0.008997,0.008892,-0.010416,-0.000709,0.906205
5,5,0.658,0.143785,-0.023998,0.105586,0.017451,0.096718,0.003267,0.077586,0.021081,...,0.010731,0.000274,0.017690,0.021904,0.020596,0.006656,0.003702,0.002842,0.010046,0.956448
6,6,0.639,0.145525,0.011598,0.073844,0.033662,0.048060,0.025851,0.062572,0.041071,...,0.008485,-0.010836,0.010704,-0.006154,0.020276,-0.001408,0.007682,-0.010400,0.008357,1.192622
7,7,0.530,0.121075,-0.009050,0.075677,0.021190,0.091292,0.031157,0.080521,0.016535,...,0.008811,0.005863,0.002372,0.020524,0.007467,0.015929,0.014293,-0.004847,0.009618,0.961146
8,8,0.591,0.138799,0.011615,0.052548,-0.000829,0.067597,0.008408,0.060088,0.001551,...,0.004362,0.018077,-0.002493,-0.000330,0.004110,0.002687,0.004334,0.005172,0.002692,0.915409
9,9,0.607,0.055749,-0.115000,0.040097,-0.075982,0.037903,-0.080345,0.012268,-0.055808,...,-0.015499,-0.002867,-0.015026,-0.008546,-0.007547,0.001105,-0.007986,-0.002980,-0.006413,0.926034
